# Use of symmetries in optical spectra from time-propagation
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/use_of_symmetries_in_optical_spectra_from_time-propagation/)

In this tutorial we will see how the spatial symmetries of a molecule can be used to reduce the number of time-propagations required to compute the absorption cross-section.

## Introduction

The dynamic polarizability (related to optical absorption cross-section via $\sigma = \frac{4 \pi \omega}{c} \mathrm{Im} \alpha $) is, in its most general form, a 3x3 tensor. The reason is that we can shine light on the system polarized in any of the three Cartesian axes, and for each of these three cases measure how the dipole of the molecule oscillates along the three Cartesian axes. This usually means that to obtain the full dynamic polarizability of the molecule we usually need to apply 3 different perturbations along $x, y, z\,$, by setting [TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection) to 1, 2, or 3.

However, if the molecule has some symmetries, it is in general possible to reduce the total number of calculations required to obtain the tensor from 3 to 2, or even 1.[$^1$](#first_reference)

To use this formalism in **Octopus**, you need to supply some extra information. The most important thing that the code requires is the information about equivalent axes, that is, directions that are related through some symmetry transformation. Using these axes, we construct a reference frame and specify it with the [TDPolarization](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarization) block. Note that these axes need not be orthogonal, but they must be linearly-independent. The [TDPolarizationEquivAxes](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationequivaxes) tells the code how many equivalent axes there are. Ideally, the reference frame should be chosen to maximize the number of equivalent axes. When using three equivalent axes, an extra input variable, [TDPolarizationWprime](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationwprime), is also required.

Let us give a couple of examples, which should make all these concepts easier to understand.


In [ ]:
import pandas as pd

In [ ]:
!mkdir -p 6_Use_of_symmetries_in_optical_spectra_from_time_propagation

In [ ]:
cd 6_Use_of_symmetries_in_optical_spectra_from_time_propagation

## Methane

As seen in previous tutorials, the methane molecule has $T_d$ symmetry. This means that it is trivial to find three linearly-independent equivalent axes such that only one time-propagation is needed to obtain the whole tensor. As it happens, we can use the usual $x$, $y$, and $z$ directions, with all of them being equivalent (check the [symmetry operations](https://en.wikipedia.org/wiki/Symmetry_operation) of the $T_d$ [point group](https://en.wikipedia.org/wiki/Molecular_symmetry) if you are not convinced). Therefore we can perform just one propagation with the perturbation along the $x$ direction adding the following to the input file used previously:
```

%TDPolarization
 1 | 0 | 0
 0 | 1 | 0
 0 | 0 | 1
%
TDPolarizationDirection = 1
TDPolarizationEquivAxes = 3
%TDPolarizationWprime
 0 | 0 | 1
%
```

#### Groundstate

In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

In [ ]:
!octopus

#### Time-propagation

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.004/eV
TDMaxSteps = 2500  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom

%TDPolarization
 1 | 0 | 0
 0 | 1 | 0
 0 | 0 | 1
%
TDPolarizationDirection = 1
TDPolarizationEquivAxes = 3
%TDPolarizationWprime
 0 | 0 | 1
%

In [ ]:
!octopus

Note that we had omitted the blocks [TDPolarization](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarization) and [TDPolarizationWprime](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationwprime) in the previous tutorials, as these are their default.
Once the time-propagation is finished, you will find, as usual, a `td.general/multipoles` file. This time the file contains all the necessary information for **Octopus** to compute the full tensor, so running the `oct-propagation_spectrum` utility will produce two files: `cross_section_vector.1` and `cross_section_tensor`.

In [ ]:
!oct-propagation_spectrum

Try comparing the spectrum for each component of the $\sigma$ tensor.

In [ ]:
df = pd.read_csv(
    "cross_section_tensor",
    sep="\s{6}|(?<=\s)-\s?",
    engine="python",
    usecols=[0, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    names=[
        "Energy",
        "sigma(1,1,1)",
        "sigma(1,2,1)",
        "sigma(1,3,1)",
        "sigma(2,1,1)",
        "sigma(2,2,1)",
        "sigma(2,3,1)",
        "sigma(3,1,1)",
        "sigma(3,2,1)",
        "sigma(3,3,1)",
    ],
    skiprows=13,
)

df.plot(
    x="Energy",
    y=[
        "sigma(1,1,1)",
        "sigma(1,2,1)",
        "sigma(1,3,1)",
        "sigma(2,1,1)",
        "sigma(2,2,1)",
        "sigma(2,3,1)",
        "sigma(3,1,1)",
        "sigma(3,2,1)",
        "sigma(3,3,1)",
    ],
    legend=False,
    subplots=True,
    layout=(3, 3),
    figsize=(12, 12),
);

## Linear molecule

Now let us look at a linear molecule. In this case, you might think that we need two calculations to obtain the whole tensor, one for the direction along the axis of the molecule, and another for the axis perpendicular to the molecule. The fact is that we need only one, in a specially chosen direction, so that our field has components both along the axis of the molecule and perpendicular to it. Let us assume that the axis of the molecule is oriented along the $x\,$ axis. Then we can use

```
 %TDPolarization
  1/sqrt(2) | -1/sqrt(2) | 0
  1/sqrt(2) |  1/sqrt(2) | 0
  1/sqrt(2) |  0         | 1/sqrt(2)
 %
 TDPolarizationDirection = 1
 TDPolarizationEquivAxes = 3
 %TDPolarizationWprime
  1/sqrt(2) |  0         | 1/sqrt(2)
 %
```

You should try to convince yourself that the three axes are indeed equivalent and linearly independent. The first and second axes are connected through a simple reflection in the $xz$ plane, transforming the $y$ coordinate from $-1/\sqrt{2}$ into $1/\sqrt{2}$. [TDPolarizationWprime](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationwprime) should be set to the result obtained by applying the inverse symmetry operation to the third axis. This actually leaves the third axis unchanged.

## Planar molecule

Finally, let us look at a general planar molecule (in the $xy$ plane). In principle we need only two calculations (that is reduced to one if more symmetries are present like, ''e.g.'', in benzene). In this case we chose one of the polarization axes on the plane and the other two rotated 45 degrees:

<code>%<a href=TODO>TDPolarization</a><br>
  1/sqrt(2) | 0 | 1/sqrt(2)<br>
  1/sqrt(2) | 0 |-1/sqrt(2)<br>
  0         | 1 | 0<br>
 %<br>
 <a href=TODO>TDPolarizationEquivAxes</a>
</code>


In this case, we need two runs, one for [TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection) equal to 1, and another for it equal to 3. Note that if there are less than 3 equivalent axes, [TDPolarizationWprime](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationwprime) is irrelevant.


[Back to main tutorial series](../Readme.ipynb)

## References

1. M.J.T. Oliveira, A. Castro, M.A.L. Marques, and A. Rubio, On the use of Neumann's principle for the calculation of the polarizability tensor of nanostructures, [J. Nanoscience and Nanotechnology](https://doi.org/10.1166/jnn.2008.142) 8 1-7 (2008);  
<span id="first_reference"></span>